# Flap with Beams2D

In [ ]:
import beams2D as b2d
from beams2D import PrettyPrint as pp

# Read the inputs in the inputsB2D object
inputsB2D = b2d.Inputs(infile="inputs.json")

geom = b2d.Geom(inputsB2D.geom)
geom.process(printDebugMessage=0)

In [ ]:


mesh = b2d.Mesh(inputsB2D.mesh)
mesh.process(geom, printDebugMessage=0, printMeshInfo=0)
#mesh.writeVTK()

#print details of inputs
pp.printInputs(inputsB2D, 'geom', title='Geometry/Structures Inputs')
pp.printInputs(inputsB2D, 'mesh', title='Mesh Inputs')
pp.printInputs(inputsB2D, 'strucParams', title='Strucutural Properties Inputs')
pp.printInputs(inputsB2D, 'bcs', title='Boundary Conditions Inputs')

In [ ]:
# assemble the system using a given element type
sysB2D = b2d.Assemble(mesh)
# build the global stiffness matrix 
sysB2D.buildGlobalMK(inputsB2D.strucParams, elemType="EBB", lumpedMass=False)

# apply boundary conditions and get effective DOFs (also remove DOFs with all zeros K, M rows)
sysB2D.applyBoundaryConditions(inputsB2D.bcs, retainNonZeroDofsOnly=True, tol=1e-8)

In [ ]:
# Check all zero rows has been removed - for eigen vectors compuatation, otherwise reduce tol
pp.printMatrix(sysB2D.effK)

In [ ]:
modalAnalysis1 = b2d.Modal(sysB2D)
modalAnalysis1.computeModes(nModes=10)

In [ ]:
pp.printMatrix(modalAnalysis1.omega)

In [ ]:
pp.printMatrix(modalAnalysis1.eigVecEff)

In [ ]:
modalAnalysis1.plotMode(modeNum=1, scale=1)

In [ ]:
modalAnalysis1.plotMode(modeNum=2, scale=1)

### Compare with theoratical natural frequencies and mode shape
The natural frequencies are given as $Cn(EI/mL^4)^{0.5}$, where m is mass per unit length

Cn = 3.5160 for mode 1  
Cn = 22.0345 for mode 2 (mode 1st node at 0.774L)  
Cn = 61.69 for mode 3   

The mode shapes are given by
$$\Phi_n(x)= (cosh(a_n x)- cos(a_n x)) + \sigma_n(sinh(a_n x - sin(a_n x)) $$
where
$$ \sigma_n = \frac{cos(a_n L) + cosh(a_nL)}{sin(a_n L) + sinh(a_nL)} $$